#### AWS t2.xlarge
Vendor=GenuineIntel Model=Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz MHz=2300.234 bogomips=4600.15
ProcessorChips=1 PhyscalCores=4


In [1]:
using Distributed

#### Que es la Metaprogramacion?

La metaprogramacion es cuando se escribe codigo para que procese y modifique codigo en julia.

Existen 2 etapas (hay mas pero se va del scope de este notebook):
1. Etapa de parseo del codigo julia para poder ser ejecutado (es donde los errores de sintaxis aparecen), el resultado de este proceso es un arbol de sintaxis abstracto (Abstract Syntax Tree - AST).
2. Etapa de ejecucion del codigo parseado.

Lo que en julia se llama macro, los programadores de python lo llaman decorator. Estas macros pueden modificar el codigo y añadir o modificar su contenido.

Por ejemplo en este proyecto se usara @time y haremos en breve @hurray para demostrar.

@time agrega un inicio del cronometro antes de la etapa 2 y luego que se termine la etapa 2 lo frena e imprime tiempos y allocaciones de memoria.

@hurray es una macro que vamos a crear para que salga un gif de babu frik (personaje de star wars) cuando termine el codigo distribuido.

El notebook corre con IJulia que es simil a Ipython en jupyter notebooks, veremos que existen diferencias como por ejemplo no existe la IPython %magic syntax.

In [2]:
HTML("""
<head>
	<meta charset="utf-8">
	<title>Show Babu Frik </title>
	<link rel="stylesheet" href="toasty/toasty.css" />
</head>
<script src="https://code.jquery.com/jquery-1.11.2.min.js"></script>
<script src="toasty/jquery.toasty.js"></script>
""")

HTML{String}("<head>\n\t<meta charset=\"utf-8\">\n\t<title>Show Babu Frik </title>\n\t<link rel=\"stylesheet\" href=\"toasty/toasty.css\" />\n</head>\n<script src=\"https://code.jquery.com/jquery-1.11.2.min.js\"></script>\n<script src=\"toasty/jquery.toasty.js\"></script>\n")

In [ ]:
%%javascript

In [3]:
macro javascript_str(s) display("text/javascript", s); end
macro hurray()
    return :(
        javascript"""
        $(document).ready( function(){
                  $("body").toasty();
                  $("body").toasty('pop');
        });
        """
    )
end

@hurray (macro with 1 method)

#### PROJECT

In [4]:
workervec = [("montecarlo@worker_1:22",1),
             ("montecarlo@worker_2:22",1),
             ("montecarlo@worker_3:22",1),
             ("montecarlo@worker_4:22",1)]

4-element Array{Tuple{String,Int64},1}:
 ("montecarlo@worker_1:22", 1)
 ("montecarlo@worker_2:22", 1)
 ("montecarlo@worker_3:22", 1)
 ("montecarlo@worker_4:22", 1)

In [5]:
addprocs(workervec; tunnel=true)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [6]:
println("Number procs: $(nprocs())")
println("Number of workers: $(nworkers())")

Number procs: 5
Number of workers: 4


In [7]:
# esta funcion debe ser igual que la de agustin
@everywhere function MC_PI(N::Int)
    in_circle = 0
    for i = 1:N
        x, y = 2*rand()-1, 2*rand()-1
        in_circle += x^2 + y^2 <= 1.0
    end
    return in_circle
end

In [8]:
function DistributedCalc(f::Function, N::Int, workers::Int=-1)
    # f = Some function
    # n = Quantity simulations distributed
    # workers = if -1 full workers otherwise set number
    
    p = nworkers()
    if workers != -1 && workers <= p
        p = workers
    end
    #println("wokers set $(p)")
    
    # split the simulations over the workers set
    n = convert(Int64,round(N/p))
    
    n_in = @distributed (+) for i=1:p # reduction call
        f(n)
    end
    
    return n_in # Pi Aprox
end

DistributedCalc (generic function with 2 methods)

In [9]:
@time println( (4/1_000_000_000)*MC_PI(1_000_000_000))

3.1415922120000004
 11.551122 seconds (104.38 k allocations: 5.478 MiB)


In [12]:
@time println( (4/10_000_000_000)*DistributedCalc(MC_PI,10_000_000_000,-1) )
#@hurray

3.141591248
 31.149949 seconds (1.07 k allocations: 36.703 KiB)


#### Further generalization for general defined integrals

![alt text](integral_montecarlo.png "Diagram of calls")

In [ ]:
@everywhere function f1(x)
    return x^3
end

In [ ]:
## changed the arguments of pointsCircle -> x_min::Int,x_max::Int,y_min::Int,y_max::Int
#
@everywhere function integralPoint(f::Function, n::Int64, x_min::Int, x_max::Int, y_min::Int, y_max::Int)
    
    in_integral = 0
    
    for i=1:n

        x = rand()*(x_max-x_min) + x_min   # pick a random point in the range
        y = rand()*(y_max-y_min) + y_min

        if 0< y < f(x)   # if the y coordinate is between the x-axis and f(x) with f(x)>0, add it to in_integral
            in_integral += 1
        end

        if f(x) < y < 0   # if the y coordinate is between the x-axis and f(x) with f(x)<0, subtract it from in_integral
            in_integral -=1
        end
    
    end
    
    return in_integral

end

In [ ]:
## changed the arguments of DistributedCalc -> x_min::Int,x_max::Int,y_min::Int,y_max::Int
#
function DistributedCalc(f::Function, N::Int, x_min::Int, x_max::Int, y_min::Int, y_max::Int, workers::Int=-1)
    # f = Some function
    # n = Quantity simulations distributed
    # workers = if -1 full workers otherwise set number
    
    n_in = 0
    
    p = nworkers()
    if workers != -1 && workers <= p
        p = workers
    end
    # println("wokers set: $(p)")
    
    # split the simulations over the workers set
    n = convert(Int64,round(N/p))
    
    n_in = @distributed (+) for i=1:p # reduction call
        
       integralPoint(f, N, x_min, x_max, y_min, y_max)
        
    end
    
    return n_in
end

In [ ]:
function McIntegral(f::Function,N::Int64, range::Any)

    # f =  general function
    # N = maximun number of montecarlo steps
    # range = list of min and max values to define the interval

    x_min = range[1]
    x_max = range[2]

    in_integral = 0
    
    y_max = findmax(map(f,collect(x_min:x_max)))[1]   # find the max of the function in the interval
    y_min = findmin(map(f,collect(x_min:x_max)))[1]   # find the min of the function in the interval

    return ( ((x_max-x_min)*(y_max-y_min))/N) * DistributedCalc(f, N, x_min, x_max, y_min, y_max)

end


In [ ]:
## Non distributed visual calculation
function VisualMcIntegral(f::Function,N::Int,MIN::Int64, MAX::Int64)
    
    x_list_in=[]
    x_list_out=[]
    y_list_in=[]
    y_list_out=[]
    
    sizehint!(x_list_in,N)
    sizehint!(x_list_out,N)
    sizehint!(y_list_in,N)
    sizehint!(y_list_out,N)
    
    y_max = findmax(map(f,collect(MIN:MAX)))[1]
    y_min = findmin(map(f,collect(MIN:MAX)))[1]
    
    for i = 1 : N
        
        x = rand()*(MAX-MIN) + MIN
        y = rand()*(y_max-y_min) + y_min
 
        if 0 < y < f(x) || f(x) < y < 0
            push!(x_list_in,x)
            push!(y_list_in,y)
        
        else
            push!(x_list_out,x)
            push!(y_list_out,y)
        
        end
        
    end
    
    ci = scatter(x_list_in,y_list_in,
                leg = false,
                aspect_ratio = 1,
                markersize=[2,2], color=["blue"],xlims = (MIN,MAX), ylims = (y_min,y_max))
    

end

In [ ]:
VisualMcIntegral(f1,10_000,0,1)

In [ ]:
println( McIntegral(f1,100_000_000,[0,1]))

-----------------------------------------------